In [ ]:
# Modules to Import

import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from joblib import dump
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau

In [ ]:
# Load Dataset

train_dataset=pd.read_csv("train.csv")
train_dataset

In [ ]:
# Drop the Label Column to make training examples

X=train_dataset.drop(['label'],axis=1)

In [ ]:
# Training Labels

y=train_dataset['label']

In [ ]:
# Split dataset into train/test for evaluation

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [ ]:
# Converting pandas dataframe to numpy for keras 

x_train=x_train.to_numpy()
y_train=y_train.to_numpy()
x_test=x_test.to_numpy()
y_test=y_test.to_numpy()

In [ ]:
# Making label one_hot_encoding

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# Normalizing the data 

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [ ]:
x_train.shape

In [ ]:
callbacks=[EarlyStopping(monitor = "val_loss", patience = 15, 
            restore_best_weights = True, mode='min')
            ,ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, mode='min')
            ]

In [ ]:
# Model structure and hyperparameters

batch_size = 32
num_classes = 10
epochs = 30
model = Sequential()
model.add(Dense(512,activation='relu'))
# model.add(Dense(256,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adam(learning_rate=0.001),metrics=['accuracy'])
hist = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test),callbacks=callbacks)
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# Saving the model so that it can be used later

dump(model,'recognize.joblib')